In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from algorithms import get_model
from utils.data import Data


In [3]:
CONFIG_PATH = "./experiments/lightgbm0.yaml"
with open(CONFIG_PATH, 'r') as f:
    config = yaml.load(f, Loader=yaml.Loader)
config

{'name': 'lightgbm0',
 'algo': 'lightgbm',
 'weights_path': './weights/lightgbm0.pkl',
 'model_params': {'boosting_type': 'gbdt',
  'num_leaves': 50,
  'max_depth': -1,
  'learning_rate': 0.2,
  'n_estimators': 200,
  'objective': 'MAE'},
 'hyper_search': {'num_leaves': ['int', 2, 31],
  'n_estimators': ['int', 50, 300],
  'max_depth': ['int', 3, 8],
  'min_child_samples': ['float', 100, 1200],
  'learning_rate': ['float', 0.0001, 0.99],
  'min_data_in_leaf': ['float', 5, 90],
  'bagging_fraction': ['float', 0.0001, 1.0],
  'feature_fraction': ['float', 0.0001, 1.0]},
 'data': {'ips_file': './data/OxCGRT_latest.csv',
  'nb_lookback_days': 1,
  'y': 'NewCasesRM'}}

In [4]:
data = Data(config)

/home/schapke/miniconda2/envs/xprize/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Warning! Interpolating missing ConfirmedCases statistics. (This will affect the ground truth labels NewCases).


In [29]:
X, y, info = data.build_train()

In [26]:
X.head()

NewCasesRM_shift1  NewCasesRMGrowthRM_shift1  \
Date       GeoID                                                            
2020-01-09 Afghanistan__nan                0.0                        0.0   
           Albania__nan                    0.0                        0.0   
           Algeria__nan                    0.0                        0.0   
           Andorra__nan                    0.0                        0.0   
           Angola__nan                     0.0                        0.0   

                             AccCasesRM_shift1  VarAccCasesRM_shift1  \
Date       GeoID                                                       
2020-01-09 Afghanistan__nan                0.0                   0.0   
           Albania__nan                    0.0                   0.0   
           Algeria__nan                    0.0                   0.0   
           Andorra__nan                    0.0                   0.0   
           Angola__nan                     0.0                   0.0   

                             C1_School closing_Diff4  \
Date       GeoID                                       
2020-01-09 Afghanistan__nan                      0.0   
           Albania__nan                          0.0   
           Algeria__nan                          0.0   
           Andorra__nan                          0.0   
           Angola__nan                           0.0   

                             C2_Workplace closing_Diff4  \
Date       GeoID                                          
2020-01-09 Afghanistan__nan                         0.0   
           Albania__nan                             0.0   
           Algeria__nan                             0.0   
           Andorra__nan                             0.0   
           Angola__nan                              0.0   

                             C3_Cancel public events_Diff4  \
Date       GeoID                                             
2020-01-09 Afghanistan__nan                            0.0   
           Albania__nan                                0.0   
           Algeria__nan                                0.0   
           Andorra__nan                                0.0   
           Angola__nan                                 0.0   

                             C4_Restrictions on gatherings_Diff4  \
Date       GeoID                                                   
2020-01-09 Afghanistan__nan                                  0.0   
           Albania__nan                                      0.0   
           Algeria__nan                                      0.0   
           Andorra__nan                                      0.0   
           Angola__nan                                       0.0   

                             C5_Close public transport_Diff4  \
Date       GeoID                                               
2020-01-09 Afghanistan__nan                              0.0   
           Albania__nan                                  0.0   
           Algeria__nan                                  0.0   
           Andorra__nan                                  0.0   
           Angola__nan                                   0.0   

                             C6_Stay at home requirements_Diff4  ...  \
Date       GeoID                                                 ...   
2020-01-09 Afghanistan__nan                                 0.0  ...   
           Albania__nan                                     0.0  ...   
           Algeria__nan                                     0.0  ...   
           Andorra__nan                                     0.0  ...   
           Angola__nan                                      0.0  ...   

                             C3_Cancel public events  \
Date       GeoID                                       
2020-01-09 Afghanistan__nan                      0.0   
           Albania__nan                          0.0   
           Algeria__nan                          0.0   
           Andorra__nan        

In [28]:
X.columns

[autoreload of utils.data failed: Traceback (most recent call last):
  File "/home/schapke/miniconda2/envs/xprize/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/home/schapke/projects/xprize/pandemic_response/work/utils/data.py", line 100
    pass
       ^
IndentationError: expected an indented block
]


Index(['NewCasesRM_shift1', 'NewCasesRMGrowthRM_shift1', 'AccCasesRM_shift1',
       'VarAccCasesRM_shift1', 'C1_School closing_Diff4',
       'C2_Workplace closing_Diff4', 'C3_Cancel public events_Diff4',
       'C4_Restrictions on gatherings_Diff4',
       'C5_Close public transport_Diff4', 'C6_Stay at home requirements_Diff4',
       'C7_Restrictions on internal movement_Diff4',
       'C8_International travel controls_Diff4',
       'H1_Public information campaigns_Diff4', 'H2_Testing policy_Diff4',
       'H3_Contact tracing_Diff4', 'H6_Facial Coverings_Diff4', 'NPISum',
       'NPISum-Diff2', 'NPISum-Diff4', 'NPISum-Diff8', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing', 'H6_Fac

In [27]:
y.head()

Date        GeoID           
2020-01-09  Afghanistan__nan    0.0
            Albania__nan        0.0
            Algeria__nan        0.0
            Andorra__nan        0.0
            Angola__nan         0.0
Name: NewCasesRM, dtype: float64

In [ ]:
def plot_country(geo_id):
    fig = plt.figure(figsize=(16,10))
    grid = GridSpec(nrows=2, ncols=1, height_ratios=[2, 1])
    ax1 = fig.add_subplot(grid[0])
    ax2 = fig.add_subplot(grid[1])
    
    gdf = df[df.GeoID == geo_id]
    gdf.NewCases.plot(ax=ax1, title=geo_id)
    gdf.NewCases.rolling(window=10).mean().plot(ax=ax1)
    gdf.NewCases.rolling(window=14, center=True, min_periods=4).mean().plot(ax=ax1)
    
    #gdf.GovernmentResponseIndex.plot(ax=ax2)
#    gdf[["C1_School closing", "C2_Workplace closing", 
#         "C5_Close public transport", "C6_Stay at home requirements", 'C3_Cancel public events', 
#         'C4_Restrictions on gatherings', "C8_International travel controls", 
#         "C7_Restrictions on internal movement"]].plot(ax=ax2, kind='line')
    #gdf[['StringencyIndex']].plot(ax=ax2, kind='line')
    gdf[['ConfirmedDeaths']].plot(ax=ax2, kind='line')
    plt.show()
    #print(gdf.columns)

In [ ]:
geo_ids = df.GeoID.unique()
for i, geo_id in enumerate(geo_ids):
    if i > 100 and i < 101:
        plot_country(geo_id)